In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [4]:
from sagemaker.tensorflow import TensorFlow

hyperparams={'epochs'              : 1,
             'layers'              : 'heads', # heads or all
             'pretrained_weight'   : 'imagenet', # imagenet, coco
            }

bucket_name = sagemaker_session.default_bucket()
output_path = f's3://{bucket_name}/jobs'

In [5]:
import os

os.listdir("train/")

['.ipynb_checkpoints',
 'annotations.json',
 'requirements.txt',
 'Mask_RCNN',
 'model.py',
 'utils.py',
 'preprocessor.py',
 'train.py']

In [11]:
tf_estimator = TensorFlow(entry_point          = 'train.py', 
                          source_dir           = 'train/',
                          output_path          = f'{output_path}/',
                          code_location        = output_path,
                          role                 = role,
                          instance_count       = 1, 
                          instance_type        = 'ml.p3.2xlarge',
                          framework_version    = '1.15.3', 
                          py_version           = 'py3',
                          script_mode          = True,
                          sagemaker_session    = sagemaker_session,
                          hyperparameters      = hyperparams)

In [ ]:
import time
job_name=f'tensorflow-maskrcnn-gpu-{time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())}'
tf_estimator.fit({'training'  : "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_train",
                  'validation': "s3://sagemaker-us-east-2-186402456289/sagemaker/mask_rcnn_localization_valid",
                  }, 
                 job_name = job_name)

2021-08-25 00:32:07 Starting - Starting the training job...
2021-08-25 00:32:30 Starting - Launching requested ML instancesProfilerReport-1629851516: InProgress
......
2021-08-25 00:33:33 Starting - Preparing the instances for training.........
2021-08-25 00:34:54 Downloading - Downloading input data...
2021-08-25 00:35:31 Training - Downloading the training image......
2021-08-25 00:36:32 Training - Training image download completed. Training in progress.WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.

2021-08-25 00:36:26,672 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-08-25 00:36:29,595 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/usr/bin/python3 -m pip install -r requirements.txt
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp36-cp36m